In [1]:
import datetime
import json
import os
#import time
import websocket
#import whisper
from dotenv import load_dotenv
from datetime import datetime
from raptorRag import RaptorRagPipeline
import summarizeContext
import contextAnalizer
from langchain_core.runnables import RunnableLambda, RunnableParallel

load_dotenv('.env.private', override=True)
api_key = os.getenv('API_KEY')
#model = RaptorRagPipeline(api_key, 'model/RaptorDB.pickle') # для локальной работы
model = RaptorRagPipeline(api_key, 'RaptorDB.pickle')

raptor_runnable = RunnableLambda(lambda text: model.answer_question(text))

#/// Добавить в пайп по ходу диалога
intent_class = contextAnalizer.IntentClassifier(api_key=api_key)
intent_runnable = RunnableLambda(lambda text: intent_class.intent_classifier(user_input=text))

alignment_class = contextAnalizer.Alignmnet(api_key=api_key)
aligment_runnable = RunnableLambda(lambda text: alignment_class.alignment_classifier(question=text))

suggestion = contextAnalizer.ContextRAG(api_key=api_key, save_path='ML_Prekoli/faiss_index')
suggestion_runnable = RunnableLambda(lambda text: suggestion.get_top_3(input_question=text))
#/// Добавить в пайп по ходу диалога

2025-04-21 05:01:43,781 - Loading faiss with AVX2 support.
2025-04-21 05:01:43,890 - Successfully loaded faiss with AVX2 support.
2025-04-21 05:01:43,897 - Failed to load GPU Faiss: name 'GpuIndexIVFFlat' is not defined. Will not load constructor refs for GPU indexes.
2025-04-21 05:01:44,890 - Successfully initialized TreeBuilder with Config 
        TreeBuilderConfig:
            Tokenizer: <Encoding 'cl100k_base'>
            Max Tokens: 100
            Num Layers: 5
            Threshold: 0.5
            Top K: 5
            Selection Mode: top_k
            Summarization Length: 100
            Summarization Model: <raptorRag.CustomSummarizationModel object at 0x10e485010>
            Embedding Models: {'EMB': <raptorRag.CustomEmbedder object at 0x1431cc4d0>}
            Cluster Embedding Model: EMB
        
        Reduction Dimension: 10
        Clustering Algorithm: RAPTOR_Clustering
        Clustering Parameters: {}
        
2025-04-21 05:01:44,891 - Successfully initialized Cl

In [2]:
prompt_pipeline = RunnableParallel(
    intent=intent_runnable,
    alignment=aligment_runnable,
    suggestion=suggestion_runnable,
)

In [3]:
raptor_runnable.invoke("Чем отличаеся КАСКО от ОСАГО ?")

2025-04-21 05:01:45,144 - Using collapsed_tree


'КАСКО — добровольное страхование автомобиля, которое покрывает убытки владельцу даже в случае его вины. ОСАГО — обязательное страхование, защищающее интересы пострадавшей стороны в случае аварии.'

In [4]:
# Example usage
def process_user_prompt(user_input: str):
    results = prompt_pipeline.invoke(user_input)
    return results

In [5]:
process_user_prompt("Чем отличается КАСКО от ОСАГО ?")

{'intent': 'technical',
 'alignment': 'neutral',
 'suggestion': ['В чем различие между ОСАГО и КАСКО?',
  'Для чего нужно КАСКО?',
  'Что покрывает КАСКО']}

In [6]:
summary_agent = summarizeContext.ContextSummarizer(api_key=api_key)
summary_runnable = RunnableLambda(lambda chat_dict: summary_agent.summarize_dialogue(chat_history=chat_dict))
resolution_runnable = RunnableLambda(lambda chat_dict: summary_agent.resolution(chat_history=chat_dict))
crm_runnable = RunnableLambda(lambda chat_dict: summary_agent.fill(chat_history=chat_dict))

quality_assurance = summarizeContext.QualityAssurance(api_key=api_key)
quality_runnable = RunnableLambda(lambda chat_dict: quality_assurance.suggestion(chat_history=chat_dict))


In [7]:
system_prompt = f"""Ты ассистент для нахождения ответа на вопрос пользователя
Твоя задача - внимательно проанализировать запрос пользователя и найти информацию в предоставленном тексте
Постарайся отвечать кратко"""
user_prompt = f"""Информация: Вы можете посмотреть доступные опции и подключить их в разделе Каталог > Услуги > Возможности при нуле Как проверить баланс и пакеты по тарифу\n\nТекст описывает, как проверить остаток интернет-пакета, дни до его обновления и условия тарифа на главном экране. Изменения условий оплаты или услуги могут произойти после бесплатного периода. Стоимость пакета списывается при подключении. Подробности тарифа, включая его название, размер пакетов и остатки, доступны\n\n Поэтому трафик по подписке может быть не виден сразу, но он точно есть Как это выглядит: в приложении Мой МТС в Личном кабинете В приложении и в Личном кабинете можно нажать на трафик, чтобы увидеть все подключённые пакеты\n\nразмер и остатки пакетов интернета, минут, СМС  Чтобы открыть полное описание своего тарифа:   1  На главной странице нажмите на его название   2  На открывшемся экране:\n\nВ разделе "Услуги" каталога можно подключить опции и проверить баланс. Нажмите на название тарифа для уточнения платы и даты списания. Кнопка "Настроить" доступна для тарифов с пакетами. Полное описание тарифа открывается при нажатии "О тарифе". Стоимость услуг и платные оп\n\nКак узнать подробности о своём тарифе 1  На главном экране вы можете узнать: название своего тарифа размер пакетов интернета, минут, СМС и их остатки\n\nПроверить подаренные минуты можно в Личном кабинете  В Мой МТС отображаются минуты, которые входят в ваш тариф Информация в виджетах С помощью виджетов приложения Мой МТС можно:  уточнить абонентскую плату и баланс"""
assistant_response = 'Для проверки баланса и пакетов по тарифу:\n\n1. В приложении "Мой МТС" или Личном кабинете нажмите на название тарифа на главной странице.\n2. На открывшемся экране можно увидеть:\n   - Название тарифа\n   - Размер и остатки пакетов интернета, минут, SMS\n   - Подаренные минуты (доступны в Личном кабинете)\n3. Для получения полной информации о тарифе нажмите "О тарифе".\n4. Абонентская плата и баланс можно проверить через виджеты приложения.'
chat_history = [
    #{"role":"system", "content":system_prompt},
    {"role":"user", "content": user_prompt},
    {"role":"assistant", "content": assistant_response}
]

In [8]:
finish_pipeline = RunnableParallel(
    summary=summary_runnable,
    resolution=resolution_runnable,
    crm=crm_runnable,
    quality=quality_runnable
)

In [9]:
finish_pipeline.invoke(chat_history)

{'summary': '### Суммаризация:\n1. **Вопрос оператора:** Как проверить баланс и пакеты по тарифу?  \n   **Ответ ассистента:** Указаны шаги для проверки через приложение "Мой МТС" и Личный кабинет, а также информация о том, как найти полное описание тарифа.  \n   **Корректность ответа:** Да, информация предоставлена полно и понятно.\n\n2. **Вопрос оператора:** Как подключить опции и проверить их баланс?  \n   **Ответ ассистента:** Указано, что опции можно подключить в разделе "Услуги" каталога, а также как проверить баланс через приложение.  \n   **Корректность ответа:** Да, информация предоставлена полно и понятно.\n\n### Вердикт:\nАссистент корректно ответил на все вопросы оператора, предоставив полную и понятную информацию о проверке баланса, пакетов и доступных опций.',
 'resolution': 'info_provided',
 'crm': {'issue_type': 'billing',
  'client_sentiment': 'neutral',
  'resolution': 'info_provided'},
 'quality': '### Рекомендации:\n1. **Используйте дружелюбный тон**: Избегайте катег